# Acordo de chaves para comunicação confidencial e autenticada

## Connection

In [5]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        self.lproc = Process(target = left, args=(left_end,))
        self.rproc = Process(target = right, args=(right_end,))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

## Emitter

In [3]:
def Emitter(conn):
    pwd = getpass('Emitter password: ')
    try:
        #TODO: key exchange DH
        iv = os.urandom(12)
        #TODO: AES in CFB
        #TODO: hash tweak . SHA256
        cipher = Cipher(algorithms.AES(key),modes.GCM(iv),backend).encryptor()

        header = 'msgheader'.encode("utf-8")
        content = 'Here is a sample message for testing'
        #TODO: XOR w/ msg & hash tweak 
        cipher.authenticate_additional_data(header)
        content = cipher.update(content.encode("utf-8")) + cipher.finalize()
        #TODO: XOR w/ cryptogram & hash tweak
        payload = {'header': header, 'content': content, 'iv':iv, 'tag': cipher.tag}

        conn.send(payload)
        conn.close()
    except:
        print("Emitter: Error")

In [6]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.ciphers import algorithms, modes

def Receiver(conn):
    pwd = getpass('Reciever password: ')
    try:
        #TODO: key exchange
        payload = conn.recv()
        iv = payload['iv']
        m_hmac = payload['m_hmac']
        header = payload['header']
        ctxt = payload['content']
        if verifyHMAC(h_key, m_hmac, header+ctxt):
            decryptor = Cipher(algorithms.AES(key),modes.GCM(iv, tag),backend).decryptor()
            msg = extractMessage(c_key, ctxt, tweak_h)
            print("Received: " + msg.decode("utf-8"))
        conn.close()
    except:
        print("Receiver: Erro na decifragem")

def verifyHMAC(key, tag, ctxt):
    """ Computes the HMAC of a given cipher text and checkes whether it matches with
        the given tag i.e. validates the authenticity and integrity of the message
    """
    try:
        h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
        h.update(ctxt)
        h.verify(tag)                        #exception on MAC failure
        return True
    except:
        return False

def extractMessage(key, iv, tweak_h, ctxt):
    """ Extracts the plain text from a cryptogram resulting from a tweakable cipher
        where ptxt = dec(ctxt ^ hash(tweak)) ^ hash(tweak)
    """
    decryptor = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend()).decryptor()
    ptxt = (decryptor.update(ctxt ^ tweak_h) + decryptor.finalize()) ^ tweak_h
    return ptxt

O estabelecimento de uma conexão entre os dois intervenientes (`Emitter` e `Receiver`)  é feita invocando o método `auto` da classe `Connection` que cria uma comunicação síncrona entre ambos. 
Este sincronismo é exigido pelo protocolo de troca de chaves (Diffie Hellman)